# Sentiment Analysis Using Hugging Face on AWS sagemaker

In [2]:
!pip install transformers

In [3]:
!pip install -U sagemaker

INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 207.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 213.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 173.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 110.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 178.7 M

In [1]:
!pip install "sagemaker<3.0.0"

In [4]:
# WARNING: This snippet is not yet compatible with SageMaker version >= 3.0.0.
# To use this snippet, install a compatible version:
# pip install 'sagemaker<3.0.0'
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'cardiffnlp/twitter-xlm-roberta-base-sentiment',
	'HF_TASK':'text-classification'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.51.3',
	pytorch_version='2.6.0',
	py_version='py312',
	env=hub,
	role=role, 
)


In [5]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)

predictor.predict({
	"inputs": "I like you. I love you",
})

------!

[{'label': 'positive', 'score': 0.8662142157554626}]

In [9]:
# Change "input" to "inputs"
predictor.predict({
    "inputs": "I hate donal trump"
})

[{'label': 'negative', 'score': 0.942926287651062}]

In [11]:
# Change "input" to "inputs"
predictor.predict({
    "inputs": "I think you are joking trust me i am far dangerious about joking."
})

[{'label': 'negative', 'score': 0.5548674464225769}]

In [12]:
# Change "input" to "inputs"
predictor.predict({
    "inputs": "Are you serious?"
})

[{'label': 'negative', 'score': 0.7044763565063477}]